In [ ]:
# #ephemeris sim
# using LinearAlgebra
# #make sure to cite this package
# using SPICE
# using Downloads: download 
# using DifferentialEquations
# using Plots

In [ ]:
using DifferentialEquations
using LinearAlgebra
using Plots
using SPICE

In [ ]:
# using Pkg
# Pkg.activate()

In [ ]:
using DelimitedFiles
xtraj  = readdlm("xtraj.txt", '\t', Float64, '\n')

#in days
time_steps = readdlm("time.txt", '\t', Float64, '\n')
#period 

T_periodic_n = 3.414975410587678

In [ ]:
xtraj

In [ ]:
# Load leap seconds kernel
furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/naif0012.tls")

# laptop 
# furnsh("/home/fausto/naif0012.tls")

In [ ]:
# Convert the calendar date to ephemeris seconds past J2000

#was 2018
#et = utc2et("2018-02-06T20:45:00")


#et = utc2et("2000-08-06T20:45:00")

et = 0

In [ ]:
# Load a planetary ephemeris kernel
furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/de440.bsp")

#laptop
# furnsh("/home/fausto/de440.bsp")

In [ ]:
#gravitational parameters for the bodies (km3/s2)

#moon 
# μ_m = 4902.80058214776

# #earth
# μ_e = 398600.432896939

# #sun
# μ_s = 132712440017.987

In [ ]:
#units used to normalize
#also in km and seconds

#distance
# l_star = 385692.50

# #time
# t_star = 377084.152667038

# #in km
# lunar_radius = 1738.2 

In [ ]:
# μ_earth_moon = 1.215058560962404E-2

In [ ]:
G = 6.67430e-11/(1000)^3 #units: km3/kg s2 
earth_mass = 5.97219e24
moon_mass = 7.34767309e22
μ_gm = G*(earth_mass+moon_mass)

In [ ]:
μ_e = 3.986004415000000e5
μ_m = 4.902800582147800e3

#from paper
# l_star = 3.84400e5

# μ_earth_moon = 1.215058535056245e-2

# t_star = 3.751902588926273e5

In [ ]:
#from the cr3bp book
l_star = 3.850e5

μ_earth_moon = 1.215e-2

T_ = 2.361e6

t_star = T_/2*pi

In [ ]:
sqrt(μ_gm/l_star^3)

In [ ]:
pose_m1 = [-μ_earth_moon*l_star , 0, 0]

In [ ]:
function hat(x)

    ω_hat = [0 -x[3] x[2]; x[3] 0 -x[1]; -x[2] x[1] 0]

    return ω_hat
    
end

In [ ]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function cr3bp_to_ephem(x_r, time)

    #state in the ECI ra
    x_eci = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    moon_state = spkezr("moon",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_moon = moon_state[1:3]

    #velocity of the moon relative to Earth
    v_moon = moon_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_moon/norm(r_moon)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_moon, v_moon)/norm(cross(r_moon, v_moon))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)

    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    eci_R_r = [x̃ ỹ z̃]

    #cr3bp assumes that the rotating frame is rotating by the mean motion 

    #mean motion θ_dot
    #θ_dot = 2*pi/2.361e6

    #θ_dot = sqrt(μ_earth_moon/norm(r_moon)^3)

    θ_dot = sqrt(μ_gm/norm(l_star)^3)

    #θ_dot = sqrt(μ_gm/norm(r_moon)^3)

    #instantanous angular velocity
    #θ_dot = norm(cross(r_moon, v_moon))/(norm(r_moon)^2)
    #θ_dot = (cross(r_moon, v_moon)/(norm(r_moon)^2))[3]

    #println("theta dot 2: ", θ_dot_2)

    ω = [0, 0, θ_dot]
    ω_hat= hat(ω)
    
    #position transformation
    x_eci[1:3] = eci_R_r*(x_r[1:3] - pose_m1)

    x_eci[4:6] = eci_R_r*x_r[4:6] + eci_R_r*ω_hat*(x_r[1:3] - pose_m1)

    return x_eci
end

In [ ]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function cr3bp_to_ephem_k(x_r, time)

    #state in the ECI ra
    x_eci = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    bary_state = spkezr("earth_barycenter",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_bary = bary_state[1:3]

    #velocity of the moon relative to Earth
    v_bary = bary_state[4:6]

    θ_dot = sqrt(μ_gm/norm(l_star)^3)

    ω = [0, 0, θ_dot]

    ω_hat= hat(ω)
    
    #position transformation
    x_eci[1:3] = r_bary + x_r[1:3]

    x_eci[4:6] = v_bary + (x_r[4:6] - v_bary) + cross(ω, (x_eci[1:3] - r_bary))

    return x_eci
end

In [ ]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function ephem_to_cr3bp_k(x_eci, time)

    #state in the ECI ra
    x_r = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    bary_state = spkezr("earth_barycenter",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_bary = bary_state[1:3]

    #velocity of the moon relative to Earth
    v_bary = bary_state[4:6]

    θ_dot = sqrt(μ_gm/norm(l_star)^3)

    ω = [0, 0, θ_dot]

    ω_hat= hat(ω)
    
    #position transformation
    x_r[1:3] = x_eci[1:3] - r_bary 

    x_r[4:6] = x_eci[4:6] - cross(ω, (x_eci[1:3] - r_bary)) 
    
    return x_eci
end

In [ ]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function cr3bp_to_ephem_k2(x_r, time)

    #state in the ECI ra
    x_eci = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    bary_state = spkezr("earth_barycenter",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_bary = bary_state[1:3]

    #velocity of the moon relative to Earth
    v_bary = bary_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_bary/norm(r_bary)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_bary, v_bary)/norm(cross(r_bary, v_bary))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)
 
    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    eci_R_r = [x̃ ỹ z̃]

    θ_dot = sqrt(μ_gm/norm(l_star)^3)

    ω = [0, 0, θ_dot]

    ω_hat= hat(ω)
    
    #position transformation
    x_eci[1:3] = r_bary + eci_R_r*x_r[1:3]

    x_eci[4:6] = v_bary + (eci_R_r*ω_hat*x_r[1:3] + eci_R_r*x_r[4:6])

    return x_eci
end

In [ ]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function ephem_to_cr3bp_k2(x_eci, time)

    #state in the ECI ra
    x_r = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    bary_state = spkezr("earth_barycenter",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_bary = bary_state[1:3]

    #velocity of the moon relative to Earth
    v_bary = bary_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_bary/norm(r_bary)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_bary, v_bary)/norm(cross(r_bary, v_bary))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)
 
    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    eci_R_r = [x̃ ỹ z̃]

    θ_dot = sqrt(μ_gm/norm(l_star)^3)

    ω = [0, 0, θ_dot]

    ω_hat= hat(ω)
    
    #position transformation
    x_r[1:3] = x_eci[1:3] - r_bary 

    x_r[4:6] = x_eci[4:6] - cross(ω, (x_eci[1:3] - r_bary)) 
    
    return x_eci
end

In [ ]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function ephem_to_cr3bp(x_eci, time)

    #state in the ECI ra
    x_r = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    moon_state = spkezr("moon",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_moon = moon_state[1:3]

    #velocity of the moon relative to Earth
    v_moon = moon_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_moon/norm(r_moon)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_moon, v_moon)/norm(cross(r_moon, v_moon))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)

    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    eci_R_r = [x̃ ỹ z̃]

    r_R_eci = inv(eci_R_r) #also the transpose since it is a rotation matrix

    #cr3bp assumes that the rotating frame is rotating by the mean motion 

    #θ_dot = 2*pi/2.361e6

    #θ_dot = sqrt(μ_earth_moon/norm(r_moon)^3)

    θ_dot = sqrt(μ_gm/norm(l_star)^3)

    #θ_dot = sqrt(μ_gm/norm(r_moon)^3)

    #instantanous angular velocity
    #θ_dot = norm(cross(r_moon, v_moon))/(norm(r_moon)^2)
    #θ_dot_2 = (cross(r_moon, v_moon)/(norm(r_moon)^2))[3]

    #println("theta dot 2: ", θ_dot_2)

    ω = [0, 0, θ_dot]

    ω_hat= hat(ω)
    
    #position transformation
    x_r[1:3] = (r_R_eci*x_eci[1:3]) + pose_m1

    x_r[4:6] =  r_R_eci*(x_eci[4:6] - eci_R_r*ω_hat*(x_r[1:3] - pose_m1))

    return x_r
end

In [ ]:
test = [431249.946141646,
0.0,
2286.769716989665,
0.0,
0.1811228748758214,
0.0]

In [ ]:
#this matches the pavlack paper
ephem_sim = cr3bp_to_ephem(test, et)

In [ ]:
#this matches the pavlack paper
ephem_sim = cr3bp_to_ephem_k(test, et)

In [ ]:
# barypose = spkezr("earth_barycenter",0,"J2000","NONE","earth")[1]

# moonpose = spkezr("moon",0,"J2000","NONE","earth")[1]

In [ ]:
# barypose[4:6]/norm(barypose[4:6])

In [ ]:
# moonpose[4:6]/norm(moonpose[4:6])

In [ ]:
# barypose[1:3]/norm(barypose[1:3])

In [ ]:
moonpose[1:3]/norm(moonpose)

In [ ]:
# scatter([0], [0], [0])
# scatter!([barypose[1]], [barypose[2]], [barypose[3]])
# scatter!([moonpose[1]], [moonpose[2]], [moonpose[3]])

In [ ]:
# norm(barypose[1:3])

In [ ]:
# 1.215e-2*3.850e5 

In [ ]:
# l_star 

In [ ]:
# μ_earth_moon*l_star 

In [ ]:
ephem_sim_dimensionalized = [ephem_sim[1:3]/l_star; ephem_sim[4:6]/(l_star/t_star)]

In [ ]:
function just_dynamics_integrate(x_0, period)
    
    tspan = (0.0, period)
    prob = ODEProblem(ephem_dynamics!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-15, reltol=1e-12)
    
    return sol
    
end

In [ ]:
function just_dynamics_integrate_scaled(x_0, period)
    
    tspan = (0.0, period)
    prob = ODEProblem(ephem_dynamics_scaled!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-15, reltol=1e-12)
    
    return sol
    
end

In [ ]:
function ephemeris_model_EarthMoon(x, t)

    xdot = zeros(6)

    xdot[1:3] = x[4:6]

    #position of the spacecraft relative to the earth 
    r_qi = x[1:3]

    time = et + t

    #get positions of moon and sun relative to earth 
    rqj_moon = spkpos("moon", time, "J2000", "none", "earth")[1]
    
    rij_moon = rqj_moon- r_qi 

    #r_qi - vector that defines the position of the satellite wrt Earth
    #rqj is the position of a planetary body wrt satellite

    #just Earth and Moon
    xdot[4:6] = (-μ_e/(norm(r_qi))^3)*r_qi + μ_m.*((rij_moon/norm(rij_moon)^3)-(rqj_moon/norm(rqj_moon)^3));


    return xdot

end

In [ ]:
function ephemeris_model_EarthMoon_scaled(x, t)

    #put units on x and t

    x_units = [x[1:3]*l_star; x[4:6]*(l_star/t_star)]
    t_units = t*t_star

    xdot = zeros(6)

    xdot[1:3] = x[4:6]

    #position of the spacecraft relative to the earth. normalized
    r_qi = x_units[1:3]

    time = et + t_units

    #get positions of moon and sun relative to earth. dimensionalized by l_star

    rqj_moon = (spkpos("moon", time, "J2000", "none", "earth")[1])

    rij_moon = rqj_moon- r_qi 

    #r_qi - vector that defines the position of the satellite wrt Earth
    #rqj is the position of a planetary body wrt satellite

    #just Earth and Moon
    xdot_dim = (-μ_e/(norm(r_qi))^3)*r_qi + μ_m.*((rij_moon/norm(rij_moon)^3)-(rqj_moon/norm(rqj_moon)^3));

    xdot[4:6] = xdot_dim./(l_star/t_star^2)

    
    return xdot

end

In [ ]:
function ephem_dynamics!(du, u, p, t)
    
    #dependence on time
    du[1:6] = ephemeris_model_EarthMoon(u[1:6], t)
    
end

In [ ]:
function ephem_dynamics_scaled!(du, u, p, t)
    
    #dependence on time
    du[1:6] = ephemeris_model_EarthMoon_scaled(u[1:6], t)
    
end

In [ ]:
time_steps_s = time_steps*86400

In [ ]:
sol_scaled = just_dynamics_integrate_scaled(ephem_sim_dimensionalized, time_steps_s[end]/t_star)

In [ ]:
sol = just_dynamics_integrate(ephem_sim, time_steps_s[end])

In [ ]:
#this just tests one timestep
#sol = just_dynamics_integrate(ephem_sim, time_steps_s[2])

In [ ]:
function get_state(solution)
    
    N = size(solution.u)[1]

    all_states = zeros(6, N)

    for i=1:N
        all_states[:,i] = solution.u[i][1:6]
    end
    
    #all states and all stm are functions of t
    #solution.t is the time
    return all_states
end

In [ ]:
all_states_scaled = get_state(sol_scaled)

all_states = get_state(sol)

In [ ]:
all_states_scaled[:,end]

In [ ]:
[all_states_scaled[1:3,end]*l_star; all_states_scaled[4:6,end]*(l_star/t_star)]

In [ ]:
#scaled and unscaled give the same answer
all_states[:,end]

In [ ]:
plot(all_states[1,:], all_states[2,:], all_states[3,:])

#plot!(xtraj[1,:], xtraj[2,:], xtraj[3,:])

In [ ]:
#transfer all to cr3bp 
all_states_cr3bp = zeros(size(all_states))

for i=1:size(all_states)[2]

    all_states_cr3bp[:,i] = ephem_to_cr3bp_k(all_states[:,i], et+sol.t[i])

end

In [ ]:
# plot(all_states_cr3bp[1,:], all_states_cr3bp[2,:], all_states_cr3bp[3,:])

# plot!(xtraj[1,1:7], xtraj[2,1:7], xtraj[3,1:7])

test = plot(all_states_cr3bp[1,:], all_states_cr3bp[2,:], label = "Ephemeris Forward Rollout")

plot!(xtraj[1,:], xtraj[2,:], label = "original cr3bp orbit") 

scatter!([(1-μ_earth_moon)*l_star], [0], label = "Moon", legend=false) 



#scatter!([all_states_cr3bp[1,1]], [all_states_cr3bp[2,1]], [all_states_cr3bp[3,1]])

In [ ]:
savefig(test, "traj Plots")

In [ ]:
plot(all_states_cr3bp[1,1:5], all_states_cr3bp[2,1:5], label = "Ephemeris Forward Rollout")

plot!(xtraj[1,1:2], xtraj[2,1:2], label = "original cr3bp orbit") 

In [ ]:
# all_states[:,end]

In [ ]:
# ephem_to_cr3bp(all_states[:,end], time_steps_s[2])

In [ ]:
# ephem_to_cr3bp(all_states[:,1], 0)

In [ ]:
# scatter([all_states[1,2]], [all_states[2,2]], [all_states[3,2]])
# scatter!([check[1]], [check[2]], [check[3]])

In [ ]:
# #convert the ephemeris back to cr3bp

# for i = 1:size(all_states)[2]
#     all_states[:,i] = cr3bp_to_ephem(all_states[:,i], time_steps_s[i])
# end

In [ ]:
#tried one timestep and it is in the correct direction. (this is with the regular km s dynamics) Maybe
#do the same test with the scaled dynamics (just one step) and see if it works. Get x traj at fixed time_steps
#steps (the 41 knot points) and compare to that. 

#verify the angular velocity term



#Maybe period of integration is wrong. 
#and may work better with just fixed defined timesteps